In [154]:
pip install lxml

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 17.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#import everything we need
from music21 import * 
from music21 import corpus, meter, tempo, expressions
import numpy as np
import pandas as pd
from collections import Counter
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

SyntaxError: invalid syntax (400620156.py, line 10)

In [ ]:
#create dictionary with file names and file paths to them
folder_path = '/Users/melodyqian/Documents/GitHub/pianocompetitionpredictions/winnersample/'
paths=[]
filenames=[]
for filename in os.listdir(folder_path):
    if filename.endswith('.mxl'):  # Or the file extension you're working with
        full_path = os.path.join(folder_path, filename)
        paths.append(full_path)
        filenames.append(filename)

for i in range(len(filenames)):
    filenames[i] = filenames[i].replace('_', ' ')  
    filenames[i] = filenames[i].replace('.mxl', '')

pathsdict=dict(zip(filenames, paths))
#len(pathsdict)
#There are 114 items

114

In [ ]:
#Read in CSV with piece names. modify so it's just pieces that were found. makes matching easier for later.
winnercsv=pd.read_csv("/Users/melodyqian/Documents/GitHub/pianocompetitionpredictions/FinalistPiecesCSV.csv")
json=pd.read_json("/Users/melodyqian/Documents/GitHub/pianocompetitionpredictions/finalist_sheet_music_results.json")
valid_indices = json[json['title'].notna()].index
filteredwinnercsv = winnercsv.loc[valid_indices].copy()
filteredwinnercsv.to_csv("/Users/melodyqian/Documents/GitHub/pianocompetitionpredictions/FilteredFinalistsCSV.csv", index=False)
difficultymetric=json.dropna(subset=['title'])


157

In [ ]:
#forgot to do this to add biographical info my bad
biocsv=pd.read_csv("/Users/melodyqian/Documents/GitHub/pianocompetitionpredictions/finalistbioinfo.csv")
filteredbiocsv=biocsv.loc[valid_indices].copy()
filteredbiocsv.to_csv("filteredbio.csv")

In [18]:
def match_titles_to_files(df, filenames, dict, threshold=70):
    matched_files = []
    for title in df['title']:
        match = process.extractOne(title, filenames, scorer=fuzz.token_sort_ratio)
        if match and match[1] >= threshold: 
           matched_files.append(dict.get(match[0], np.nan))  
        else:
            matched_files.append(np.nan)  
    return matched_files  



In [98]:
#now we have the query matched matched to the title matched to the file. this way we can match the difficulty metric to the main CSV we imported earlier.
filekeys = list(pathsdict.keys())
difficultymetric['matched_file'] = match_titles_to_files(difficultymetric, filekeys, pathsdict)
difficultymetric['matched_file'] = difficultymetric['matched_file'].where(pd.notna(difficultymetric['matched_file']), None)
difficultymetric['matched_file']=difficultymetric['matched_file'].astype(str)

/var/folders/bf/j0nvm48j18b4mlfybgxf_1fm0000gn/T/ipykernel_6463/954534441.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  difficultymetric['matched_file'] = match_titles_to_files(difficultymetric, filekeys, pathsdict)
/var/folders/bf/j0nvm48j18b4mlfybgxf_1fm0000gn/T/ipykernel_6463/954534441.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  difficultymetric['matched_file'] = difficultymetric['matched_file'].where(pd.notna(difficultymetric['matched_file']), None)
/var/folders/bf/j0nvm48j18b4mlfybgxf_1fm

In [99]:
#a dictionary we use later
note_durations = {
    '32nd_and_shorter': 0.125,
    '16th': 0.25,
    'eighth': 0.5,
    'quarter': 1,
    'dotted quarter': 1.5,
    'half': 2,
    'dotted half': 3,
    'whole': 4
}



In [100]:
#LEVIATHAN MUSIC ANALYSIS FUNCTION I'M SORRY IT'S SO UGLY I DID THEM ALL SEPARATELY AND JUST COMBINED THEM LOL
def analyze(filepath):
    if filepath is None or pd.isna(filepath):
        return None
    try:   
        score=converter.parse(filepath)
        flatscore=score.flatten()
        def notecount(flatscore):
            num_notes=0
            for elem in flatscore:
                if isinstance(elem, note.Note):
                    num_notes += 1
                elif isinstance(elem, chord.Chord):
                    num_notes += len(elem.pitches)
            return(num_notes)
        notecounts=notecount(flatscore)
        chords= [element for element in flatscore if isinstance(element, chord.Chord)]
        numberchords= len(chords)   
        def chordpitchdistances(chords):
            pitchdistances= []
            for i in range(len(chords)-1):
                firstchord= chords[i]
                secondchord= chords[i+1]
                rootInterval=interval.Interval(firstchord.root(),secondchord.root())
                pitchdistances.append(rootInterval.semitones)
            return(pitchdistances)
        pitchdistances=chordpitchdistances(chords)
        chord_abs_dis= [abs(dis) for dis in pitchdistances]
        meanchorddis= np.mean(chord_abs_dis)
        def chordtimedistances(chords):
            timedistances=[]
            for i in range(len(chords)-1):
                firstchord= chords[i]
                secondchord= chords[i+1]
                time_difference = secondchord.offset - firstchord.offset
                timedistances.append(time_difference) 
            return timedistances
        chordtimedistance=chordtimedistances(chords)
        meantimedistance=np.mean(chordtimedistance)
        key=flatscore.analyze('key')
        timesignature=flatscore.getElementsByClass(meter.TimeSignature)
        timesignaturechanges=len(timesignature)
        def findtrills(score):
            trills=0
            totalbeats=0
            for element in score.notesAndRests:
                if element.expressions:  # Check if the note has expressions
                    for expr in element.expressions:
                        if isinstance(expr, expressions.Trill):  # Check if it's a trill
                            trills+= 1 
                            totalbeats+=element.quarterLength
            return [trills, totalbeats]
        trill_length=findtrills(flatscore)
        tempos= flatscore.getElementsByClass(tempo.MetronomeMark)
        sorted_tempos = sorted(tempos, key=lambda tm: tm.measureNumber)
        weighted_tempo=[]
        for i in range(len(sorted_tempos)):
            current_bpm = sorted_tempos[i].getQuarterBPM()
            current_measure = sorted_tempos[i].measureNumber
            if i < len(sorted_tempos) - 1:
                next_measure = sorted_tempos[i + 1].measureNumber
                duration = next_measure - current_measure
            else:
                # If this is the last tempo, assume it lasts to the end of the piece
                duration = score.highestTime
            weighted_tempo.append((current_bpm, duration))
        total_weight = sum(duration for _, duration in weighted_tempo)
        weighted_mean = (
        sum(bpm * duration for bpm, duration in weighted_tempo) / total_weight
        if total_weight != 0 else 0
        )
        num_measures = len(score.recurse().getElementsByClass('Measure'))
        duration_counter = Counter(note.duration.quarterLength for note in score.recurse().notes)
        def count_notes(score, duration_counter):
            note_counts= {}
            for name, duration in note_durations.items():
                if name=='32nd_and_shorter':
                    note_counts[name]= sum(count for d, count in duration_counter.items() if d<=duration)
                else:
                    note_counts[name]=duration_counter[duration]
            return note_counts
        notedict= count_notes(score,duration_counter)
        return{
            "numberofnotes": notecounts,
            "numberofchords": numberchords,
            "numberofmeasures": num_measures,
            "meandistancebetweenchords": meanchorddis,
            "meantimebetweenchords": meantimedistance,
            "key": key,
            "timechanges": timesignaturechanges,
            "trillslength": trill_length,
            "avgtempo" : weighted_mean,
            **notedict
        }
    except Exception as e:
        print(f"Error analyzing {filepath}: {e}")
        return None

In [101]:
def safe_analyze(path):
    try:
        return analyze(path)
    except Exception as e:
        print(f"Error analyzing {path}: {e}")
        return {}

results_df = difficultymetric['matched_file'].apply(safe_analyze).apply(pd.Series)

/Users/melodyqian/Library/Python/3.9/lib/python/site-packages/music21/musicxml/xmlToM21.py:1986: MusicXMLWarning: The following exception took place in m. 10 in part Piano.
  warnings.warn(
/Users/melodyqian/Library/Python/3.9/lib/python/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/melodyqian/Library/Python/3.9/lib/python/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Error analyzing /Users/melodyqian/Documents/GitHub/pianocompetitionpredictions/winnersample/Piano_Sonata_Pastorale_in_C_Major_--_James_S_Liauw.mxl: 'str' object has no attribute 'sites'
Error analyzing None: File not found or no such format found for: None
Error analyzing None: File not found or no such format found for: None
Error analyzing None: File not found or no such format found for: None
Error analyzing None: File not found or no such format found for: None
Error analyzing None: File not found or no such format found for: None
Error analyzing None: File not found or no such format found for: None
Error analyzing None: File not found or no such format found for: None
Error analyzing None: File not found or no such format found for: None
Error analyzing None: File not found or no such format found for: None
Error analyzing None: File not found or no such format found for: None
Error analyzing None: File not found or no such format found for: None
Error analyzing /Users/melodyqian

In [121]:
results_df['Composite'] = difficultymetric['query'].values
finalistsdifficultysample= pd.concat([results_df, filteredbiocsv, filteredwinnercsv], axis=1)


In [122]:
finalistsdifficultysample.to_csv("FinalistDifficultySample.csv")

In [127]:
#getting rid of NAs for the final set
naeditdf=pd.read_csv("/Users/melodyqian/Documents/GitHub/pianocompetitionpredictions/FinalistDifficultySample.csv")
naeditdf.dropna(subset=['numberofnotes'], inplace=True)

In [128]:
naeditdf.to_csv("414FINALDIFFSAMPLE.csv")

0
